In [1]:
from pathlib import Path
import sqlite3
import pandas as pd
import geopandas as gpd

# Caminho do banco de dados
DB_PATH = Path("C:/GitHub/Global Solution 1/db/gs2025.db")

conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM igv_resultado_all", conn)
conn.close()

# Normaliza os nomes das UDHs
df["nome_udh_norm"] = df["nome_udh"].str.strip().str.lower()

# Diretório base
DADOS_DIR = Path("../dados")

# Mapeia regiões e seus respectivos shapefiles de UDH
mapa_shapefiles = {
    "Belém (PA)": DADOS_DIR / "dados_belem" / "SH RM_Belem" / "RM_Belem_UDH_region.shp",
    "Belo Horizonte (MG)": DADOS_DIR / "dados_bh" / "SH RM_Belo_Horizonte" / "RM_BeloHorizonte_UDH_region.shp",
    "Baixada Santista (SP)": DADOS_DIR / "dados_bs" / "SH RM_Baixada_Santista" / "RM_Baixada_Santista_UDH_region.shp",
    "Campinas (SP)": DADOS_DIR / "dados_campinas" / "SH RM_Campinas" / "SH RM_Campinas" / "RM_Campinas_UDH_Atlas_novo_region.shp",
    "Cuiabá (MT)": DADOS_DIR / "dados_cuiaba" / "SH RM_Cuiaba" / "RM_Cuiaba_UDH_region.shp",
    "Curitiba (PR)": DADOS_DIR / "dados_curitiba" / "SH RM_Curitiba" / "RM_Curitiba_UDH_region.shp",
    "Distrito Federal (DF)": DADOS_DIR / "dados_df" / "SH RIDE_DF" / "RM_RIDE_UDH_region.shp",
    "Florianópolis (SC)": DADOS_DIR / "dados_florianopolis" / "dados_florianopolis" / "SH RM Florianópolis" / "RM_Florianopolis_por_UDH_region.shp",
    "Fortaleza (CE)": DADOS_DIR / "dados_fortaleza" / "SH RM_Fortaleza" / "RM_Fortaleza_UDH_3_region.shp",
    "Goiânia (GO)": DADOS_DIR / "dados_goiania" / "SH RM_Goiania" / "RM_Goiania_UDH_region.shp",
    "Maceió (AL)": DADOS_DIR / "dados_maceio" / "SH RM_Maceio" / "RM_Maceio_UDH_region.shp",
    "Manaus (AM)": DADOS_DIR / "dados_manaus" / "SH RM_Manaus" / "RM_Manaus_UDH_2_region.shp",
    "Natal (RN)": DADOS_DIR / "dados_natal" / "SH RM_Natal" / "RM_Natal_UDH_2_region.shp",
    "Porto Alegre (RS)": DADOS_DIR / "dados_pa" / "SH RM_Porto_Alegre" / "RM_PortoAlegre_UDH_region.shp",
    "Petrolina/Juazeiro (PE/BA)": DADOS_DIR / "dados_petrolina_juazeiro" / "dados_Petrolina_Juazeiro" / "SH RIDE Petrolina_Juazeiro" / "RIDE_Petrolina_por_UDH_region.shp",
    "Recife (PE)": DADOS_DIR / "dados_recife" / "SH RM_Recife" / "RM_Recife_UDH_region.shp",
    "Rio de Janeiro (RJ)": DADOS_DIR / "dados_rj" / "SH RM_Rio_de_Janeiro" / "RM_RiodeJaneiro_UDH_3_region.shp",
    "Salvador (BA)": DADOS_DIR / "dados_salvador" / "SH RM_Salvador" / "RM_Salvador_UDH_4_region.shp",
    "São Luís (MA)": DADOS_DIR / "dados_sl" / "SH RM_Sao_Luis" / "RM_SaoLuis_UDH_region.shp",
    "Sorocaba (SP)": DADOS_DIR / "dados_sorocaba" / "dados_sorocaba" / "SH RM Sorocaba" / "RM_Sorocaba_por_UDH_region.shp",
    "São Paulo (SP)": DADOS_DIR / "dados_sp" / "SH RM São Paulo" / "RM_SaoPaulo_UDH_region.shp",
    "Teresina/Timon (PI/MA)": DADOS_DIR / "dados_teresina_timon" / "dados_teresina_timon" / "SH RIDE Teresina_Timon" / "RIDE_Teresina_por_UDH_region.shp",
    "Vale do Paraíba (SP)": DADOS_DIR / "dados_valedoparaiba" / "SH RM_Vale_Paraiba" / "RM_Vale_Paraiba_UDH_region.shp",
    "Vitória (ES)": DADOS_DIR / "dados_vitoria" / "SH RM_Vitoria" / "RM_Vitoria_UDH_2_region.shp",
}

# Adicione as colunas 'latitude' e 'longitude' se não existirem
if "latitude" not in df.columns:
    df["latitude"] = pd.NA
if "longitude" not in df.columns:
    df["longitude"] = pd.NA

# Processar todas as regiões
total_coords = 0

for regiao, shp_path in mapa_shapefiles.items():
    if shp_path.exists():
        print(f"Processando shapefile: {shp_path}")
        gdf = gpd.read_file(shp_path)
        print(f"Colunas disponíveis no shapefile: {gdf.columns}")
        
        # Verifique se a coluna 'UDH_ATLAS' existe
        if "UDH_ATLAS" in gdf.columns:
            gdf["nome_udh_norm"] = gdf["UDH_ATLAS"].str.strip().str.lower()
            gdf["latitude"] = gdf.geometry.centroid.y
            gdf["longitude"] = gdf.geometry.centroid.x
            coords = gdf[["nome_udh_norm", "latitude", "longitude"]]
            before_merge = df["latitude"].notna().sum()
            df = pd.merge(df, coords, on="nome_udh_norm", how="left", suffixes=("", "_new"))
            df["latitude"] = df["latitude"].combine_first(df["latitude_new"])
            df["longitude"] = df["longitude"].combine_first(df["longitude_new"])
            df.drop(columns=["latitude_new", "longitude_new"], inplace=True)
            after_merge = df["latitude"].notna().sum()
            total_coords += after_merge - before_merge
        else:
            print(f"A coluna 'UDH_ATLAS' não existe no shapefile da região: {regiao}")
    else:
        print(f"Shapefile não encontrado: {shp_path}")

# Atualizar o banco
conn = sqlite3.connect(DB_PATH)
df.drop(columns=["nome_udh_norm"]).to_sql("igv_resultado_all", conn, if_exists="replace", index=False)
conn.close()

total_coords


Processando shapefile: ..\dados\dados_belem\SH RM_Belem\RM_Belem_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')


C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["latitude"] = gdf.geometry.centroid.y
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["longitude"] = gdf.geometry.centroid.x
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:70: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df["latitude"] = df["latitude"].combine_first(df["latitude_new"])
C:

Processando shapefile: ..\dados\dados_bh\SH RM_Belo_Horizonte\RM_BeloHorizonte_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_bs\SH RM_Baixada_Santista\RM_Baixada_Santista_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_campinas\SH RM_Campinas\SH RM_Campinas\RM_Campinas_UDH_Atlas_novo_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_cuiaba\SH RM_Cuiaba\RM_Cuiaba_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_curitiba\SH RM_Curitiba\RM_Curitiba_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS'

C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["latitude"] = gdf.geometry.centroid.y
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["longitude"] = gdf.geometry.centroid.x
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["latitude"] = gdf.geometry.centroid.y
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:66: UserWarning: Geometry is in a geogra

Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_florianopolis\dados_florianopolis\SH RM Florianópolis\RM_Florianopolis_por_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_fortaleza\SH RM_Fortaleza\RM_Fortaleza_UDH_3_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_goiania\SH RM_Goiania\RM_Goiania_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_maceio\SH RM_Maceio\RM_Maceio_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapef

...

Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_salvador\SH RM_Salvador\RM_Salvador_UDH_4_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CODIGO_MUN', 'NOME_MUNIC', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_sl\SH RM_Sao_Luis\RM_SaoLuis_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_sorocaba\dados_sorocaba\SH RM Sorocaba\RM_Sorocaba_por_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_sp\SH RM São Paulo\RM_SaoPaulo_UDH_region.shp


C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["latitude"] = gdf.geometry.centroid.y
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["longitude"] = gdf.geometry.centroid.x
c:\GitHub\Global Solution 1\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: ..\dados\dados_sp\SH RM São Paulo\RM_SaoPaulo_UDH_region.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:65: UserWarning: Geometr

Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'RM', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_teresina_timon\dados_teresina_timon\SH RIDE Teresina_Timon\RIDE_Teresina_por_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_valedoparaiba\SH RM_Vale_Paraiba\RM_Vale_Paraiba_UDH_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'CD_GEOCODM', 'NM_MUNICIP', 'RM', 'geometry'], dtype='object')
Processando shapefile: ..\dados\dados_vitoria\SH RM_Vitoria\RM_Vitoria_UDH_2_region.shp
Colunas disponíveis no shapefile: Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry'], dtype='object')


C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["latitude"] = gdf.geometry.centroid.y
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:66: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["longitude"] = gdf.geometry.centroid.x
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["latitude"] = gdf.geometry.centroid.y
C:\Users\Sérgio\AppData\Local\Temp\ipykernel_20968\1759685319.py:66: UserWarning: Geometry is in a geogra

np.int64(0)

In [2]:
print(df.columns)

Index(['cod_id', 'udh_atlas', 'nome_udh', 'nome_mun', 'nome_uf', 'regiao',
       'idhm', 'ren1', 'infraestrutura', 't_rmaxidoso', 'igv', 'latitude',
       'longitude', 'nome_udh_norm'],
      dtype='object')


In [3]:
print(f"Região: {regiao}")
print(gdf.columns)

Região: Vitória (ES)
Index(['UDH_ATLAS', 'REGIONAL', 'CD_GEOCODM', 'NM_MUNICIP', 'geometry',
       'nome_udh_norm', 'latitude', 'longitude'],
      dtype='object')


In [4]:
import sqlite3
import pandas as pd
from pathlib import Path

# Caminho do banco de dados
DB_PATH = Path("C:/GitHub/Global Solution 1/db/gs2025.db")

# Conectar ao banco de dados
conn = sqlite3.connect(DB_PATH)

# Listar tabelas disponíveis no banco
print("📋 Tabelas disponíveis no banco:")
tabelas = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(tabelas)

# Visualizar as primeiras linhas da tabela 'igv_resultado_all'
if "igv_resultado_all" in tabelas["name"].values:
    print("\n📄 Dados da tabela 'igv_resultado_all':")
    df = pd.read_sql("SELECT * FROM igv_resultado_all LIMIT 1000", conn)
    print(df)
else:
    print("\n❌ A tabela 'igv_resultado_all' não foi encontrada no banco de dados.")

# Fechar a conexão
conn.close()

📋 Tabelas disponíveis no banco:
                  name
0      indicadores_udh
1        igv_resultado
2  indicadores_udh_all
3    igv_resultado_all

📄 Dados da tabela 'igv_resultado_all':
           cod_id     udh_atlas                                      nome_udh  \
0    1.150080e+12  1.150080e+12                                      40 Horas   
1    1.150080e+12  1.150080e+12           40 Horas : Conjunto Antônio Queiroz   
2    1.150080e+12  1.150080e+12            40 Horas : Condomínio Vila Firenze   
3    1.150080e+12  1.150080e+12  Águas Lindas / Júlia Seffer : Rua da Cohaspa   
4    1.150080e+12  1.150080e+12          Júlia Seffer : Conjunto Júlia Seffer   
..            ...           ...                                           ...   
995  1.314481e+12  1.314481e+12                      Morro do Gama : Miguelão   
996  1.314481e+12  1.314481e+12                         Jardim Canadá Parte A   
997  1.314481e+12  1.314481e+12                         Jardim Canadá Parte B   
998

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cod_id          1000 non-null   float64
 1   udh_atlas       1000 non-null   float64
 2   nome_udh        1000 non-null   object 
 3   nome_mun        1000 non-null   object 
 4   nome_uf         1000 non-null   object 
 5   regiao          1000 non-null   object 
 6   idhm            981 non-null    float64
 7   ren1            981 non-null    float64
 8   infraestrutura  981 non-null    float64
 9   t_rmaxidoso     981 non-null    float64
 10  igv             981 non-null    float64
 11  latitude        0 non-null      object 
 12  longitude       0 non-null      object 
dtypes: float64(7), object(6)
memory usage: 101.7+ KB


In [6]:
df['latitude'].value_counts(dropna=False)

latitude
None    1000
Name: count, dtype: int64

In [8]:
df['longitude'].value_counts(dropna=False)

longitude
None    1000
Name: count, dtype: int64